In [1]:
import yaml 
import os

with open(os.path.join("..", "keys.yaml"), "r") as file:
    config = yaml.safe_load(file)
    openai_api_key = config["openai_api_key"]

In [2]:
from explingo import Narrator, Grader 

example_narratives = [
    ("(Above ground living area square feet, 1256.00, -12527.46), (Overall material and finish of the house, 5.00, -10743.76), (Second floor square feet, 0.00, -10142.29)", 
     "The house's living area size of around 1,200 sq. ft., lower quality materials (5/10), and lack of a second floor all reduce the house's value."),
    ("(Second floor square feet, 854.00, 12757.84), (Original construction date, 2003.00, 9115.72)",
     "The house's large second floor of around 850 sq. ft and recent construction date of 2003 increase its value."),
    ("(Overall material and finish of the house, 8.00, 10743.76), (Above ground living area square feet, 2000.00, 12527.46), (Second floor square feet, 1000.00, 10142.29)",
        "The house's high quality materials (8/10), large living area size of around 2,000 sq. ft., and a second floor of around 1,000 sq. ft. all increase the house's value."),
]

explanation_format = "(feature name, feature value, SHAP feature contribution)"
context = "The model predicts house prices"

narrator = Narrator(openai_api_key=openai_api_key,
                    explanation_format=explanation_format,
                    context=context,
                    sample_narratives=example_narratives)


In [5]:
explanation = "(number of rooms, 11, 7020), (fireplace, yes, 12903)"

narrative = narrator.narrate(explanation)
narrative

"The house's large number of rooms (11) and the presence of a fireplace both increase the house's value."

In [6]:
grader = Grader(openai_api_key=openai_api_key, 
                metrics="all", 
                sample_narratives=example_narratives)

grader(explanation=explanation, explanation_format=explanation_format, narrative=narrative)

accuracy        4.0
completeness    4.0
fluency         4.0
conciseness     4.0
dtype: float64